In [1]:
%load_ext autotime
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb


%matplotlib inline
matplotlib.style.use('ggplot')

pd.options.display.max_columns = 120
pd.options.display.max_rows = 1200

# Show frequency table and plot values
def showDist(variable):
    print("Frquency counts\n ",variable.iloc[:,0].value_counts() / len(variable))
    plt.hist(variable.values)
    plt.show()
    
print("Loaded all. . .")

Loaded all. . .


In [2]:
df_raw = pd.read_pickle("Files/CleanedProcessedData.pickle")
print("File loaded, shape: {0}".format(df_raw.shape))

df_NBO = pd.read_csv('Files/NBO_Launch_20180117.log', delimiter='|')
print("File loaded, shape: {0}".format(df_NBO.shape))

File loaded, shape: (1264417, 95)
File loaded, shape: (517326, 3)
time: 4.62 s


In [3]:
df_raw.head()

,p_gender,p_2nd_ind_gender,p_age_2nd,p_child_AgeRange,p_credit_card,p_bank_card,p_gas_card,p_trav_card,p_unkwn_card,p_prem_card,p_upsc_card,p_dwelling,p_Env_Issues,p_hm_purch,p_homeval,p_homevalDS,p_Internet_user,p_mail_order,p_mail_resp,p_ocu_1st,p_ocu_1stDS,p_ocu_2nd,pp_age,pp_ageDS,pp_child,pp_child_range,pp_edu,pp_eduDS,pp_hh_size,pp_income,pp_incomeDS,pp_marital,pp_maritalDS,pp_own_rent,pp_own_rentDS,pp_res_length,pp_res_lengthDS,p_race_Code,p_race_code_DS,p_language,p_language_DS,p_technology,p_green_living,Hm_Bedroom_Count,Hm_Exterior,Hm_ExteriorDS,Hm_Heat_Source,Hm_Heat_SourceDS,Hm_Lot_Sqft_Actual,Hm_Pool,Hm_Room_Count,sqft_actual,sqft_range,sqft_rangeDS,Hm_Yr_Built_Actual,Geo_Latitude,Geo_Longitude,FIPS_county_cd,match_indc_infobase,match_indc_overall,match_num_sources,Matchlev_geo_latlong,matchlev_pp_age,matchlev_pp_child,matchlev_pp_child_range,matchlev_pp_edu,matchlev_pp_hh_size,matchlev_pp_income,matchlev_pp_marital,matchlev_pp_own_rent,matchlev_pp_res_length,dma_nomail,SOHO_indicator,Afford_Lvl,Comfort_Consumption,Green_Affinity,Info_Action_Cap,Invest_Cap,Axm_Seg_ECDS,acxiom_seg,Tech_Prop_Scr,NM_COMPRESSED,Address,NOCONTACT,tx_nocontact,city,POPPCT_URBAN,Segment,division,products_FlatBill,products_Paperless,products_BudgetBill,products_EFT,products_WaterHeater,products_HeatPump
ky_ba,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1371466055.0,0.909091,0.000000,0.486955,0.0,0.107143,1.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.878767,0.142857,0.052632,1.0,1.0,1.0,0.0,0.0,0.0,0.172840,0.170732,1.0,0.022260,0.000000,1.000000,0.250,0.750,0.750,0.666667,0.666667,1.0,1.0,0.933333,0.333333,1.0,1.00,1.0,0.75,0.157895,0.0,0.022880,0.561873,0.000000,0.089479,0.000000,0.000044,0.0,0.006052,0.000000,0.000000,0.000000,0.000000,0.181772,0.378416,0.181818,1.0,1.0,0.450000,0.4,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.4,0.2,0.2,0.4,0.6,0.750000,0.833333,0.8,0.099362,0.986745,0.0,0.0,0.981530,0.266386,0.347826,0.4,0.0,0.0,0.0,0.0,0.0,0.0
1368424276.0,0.909091,0.000000,0.486955,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.0,0.878767,0.380952,0.210526,0.0,0.0,0.0,0.0,0.0,0.0,0.123457,0.121951,0.0,0.000000,0.000000,1.000000,0.000,0.250,0.250,1.000000,1.000000,1.0,1.0,0.000000,1.000000,0.5,0.25,1.0,0.75,0.052632,0.0,0.022880,0.561873,0.000000,0.089479,0.000000,0.000198,0.0,0.006052,0.000000,0.000000,0.000000,0.000000,0.301930,0.383360,0.612702,1.0,1.0,0.516667,0.4,1.0,1.0,0.468442,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.2,0.6,0.0,0.0,0.0,0.916667,0.916667,1.0,0.070372,0.735994,0.0,0.0,0.678100,0.826106,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1354392083.0,1.000000,0.000000,0.486955,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.0,0.878767,0.142857,0.052632,1.0,0.0,0.0,0.0,0.0,0.0,0.074074,0.073171,0.0,0.000000,0.000000,1.000000,0.000,0.000,1.000,1.000000,1.000000,1.0,1.0,0.000000,1.000000,0.5,0.25,1.0,0.75,0.105263,0.0,0.022880,0.561873,0.000000,0.089479,0.000000,0.000198,0.0,0.006052,0.000000,0.000000,0.000000,0.000000,0.254228,0.460598,0.612702,1.0,1.0,0.316667,0.4,1.0,1.0,0.468442,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.916667,0.916667,1.0,0.550502,0.652602,0.0,0.0,0.058047,0.805035,1.000000,0.8,0.0,0.0,0.0,0.0,0.0,0.0
1374479198.0,0.000000,0.000000,0.486955,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.0,0.878767,0.333333,0.157895,0.0,0.0,0.0,0.0,0.0,0.0,0.197531,0.195122,0.0,0.000000,0.666667,0.666667,0.125,0.250,0.250,1.000000,1.000000,1.0,1.0,0.000000,1.000000,0.5,0.25,1.0,0.75,0.263158,0.0,0.022880,0.561873,0.000000,0.089479,0.000000,0.000198,0.0,0.006052,0.000000,0.000000,0.000000,0.000000,0.106406,0.361095,0.612702,1.0,1.0,0.516667,0.4,1.0,1.0,0.468442,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.4,0.6,0.2,0.0,0.833333,0.500000,0.8,0.639236,0.774929,0.0,0.0,0.617414,0.886991,0.739130,0.4,0.0,1.0,0.0,0.0,0.0,0.0
1360260083.0,1.000000,0.888889,0.296296,0.0,0.053571,0.0,0.0,0.0,1.0,0.0,0.0,1.00,0.0,0.965320,0.476190,0.315789,0.0,1.0,1.0,0.0,0.0,0.0,0.370370,0.365854,1.0,0.006849,0.000000,1

time: 125 ms


In [4]:
df_NBO.head()

,,,2017-10-17 11:13:44,sabayles,99674-32023
2017-10-1711:15:51,sabayles,9967432023,Paperless Billing eBill,N,"Bayles,Stephanie"
2017-10-17 11:24:17,sabayles,34352-56057,NaN,NaN,NaN
2017-10-17 11:24:47,sabayles,06591-97092,NaN,NaN,NaN
2017-10-17 11:25:07,sabayles,62152-63028,NaN,NaN,NaN
2017-10-17 11:25:28,sabayles,56702-76137,NaN,NaN,NaN


time: 15.6 ms


In [ ]:
# LGBM loop
import datetime

def trainLGBM(df,targetFeature):
    
    #dfReady = df.sample(frac=1, replace=False) 
    #dfReady = df.copy()
    
    X = dfReady.iloc[:,:-6]
    y = dfReady.loc[:,targetFeature]

    test_size = .33
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=123)
    
    print("Training on {0}".format(targetFeature))
    #print("Class balance:")
    accuBase = (y.value_counts() / len(y))[0]
    #print(accuBase)
    #print("Using {0} of the data, with a test ratio of {1}\n\n".format(sampleRatio,test_size))

    params2 = {}
    params2['learning_rate'] = .018704 # 0.011011
    params2['boosting_type'] = 'gbdt'
    params2['objective'] = 'binary'
    params2['metric'] = 'accuracy'
    params2['sub_feature'] = .514492
    params2['num_leaves'] = 255
    params2['max_depth'] = 7
    params2['min_data'] = 32
    params2['verbosity'] = 0
    params2['bagging_fraction'] = 0.85
    params2['lambda_l1'] = .018953
    params2['lambda_l2'] = .05242
    params2['bagging_freq'] = 5
    params2['nthread'] = 16

    # dont use crashes
    gbm = lgb.LGBMClassifier(**params2)

    # Fit
    gbm.fit(X_train, y_train.values.ravel())

    # Predict
    predictions = gbm.predict(X_test)
          
    accuScore = accuracy_score(y_test.values.ravel(),predictions)
    print("Accuracy: {0}".format(accuScore))
    return accuBase,accuScore    
    
    
          
targets = ['products_FlatBill','products_Paperless','products_BudgetBill','products_EFT',
           'products_WaterHeater','products_HeatPump']

scores = []
for target in targets:
    base,score = trainLGBM(df=dfReady,targetFeature=target)
    scores.append([target,base,score,(score-base),datetime.datetime.now()])
    if target == targets[-1]:
        print("Appending performance data to a DataFrame. . .")
        scoresDF = pd.DataFrame(scores, 
                                columns=['Target','Baseline','Accuracy','Lift','SysTime'])
